# Part 1

## 1.1

In [2]:
#Create text file with links (EXECUTE THE NEXT 2 CELLS ONLY ONCE)
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

anime = []

for page in tqdm(range(0, 400)):
    url = 'https://myanimelist.net/topanime.php?limit=' + str(page * 50)
    response = requests.get(url)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    for tag in soup.find_all('tr'):
        links = tag.find_all('a')
        for link in links:        
            if type(link.get('id')) == str and len(link.contents[0]) > 1:
                anime.append((link.contents[0], link.get('href')) )


100%|██████████| 400/400 [05:13<00:00,  1.27it/s]


In [3]:
f = open("links.txt", "w")
for el in anime:
    f.write(str(el[1])+'\n')
f.close()
#We get 19083 rows, which implies we have less than 20000 animes

## 1.2

In [1]:
#Download the pages
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from random import randint
#multiprocessing library

#first open file from which we get links
f = open("links.txt","r")
lines = f.readlines()

for page in tqdm(range(364,382)):
    #Manage the directories
    os.system("mkdir Page"+str(page+1)) #the (page+1) is to make so we don't have the first element 0 but 1
    os.chdir("Page"+str(page+1))
    
    #These if conditions are needed to have the right indexing of the pages downloaded
    start=50*page
    
    for i in range(start,start+50):
        url=lines[i]
        check = requests.get(url)
        
        #Check if the site is blocking the download, start again when we are allowed to
        if check.status_code != 200:
            print("Last well executed page is {}".format(page-1))
            while(check.status_code != 200): #Stop while we can't download
                check = requests.get(url)
                #next line was just a try of changing the ip when the site would stop the downloads
                #in short didn't manage to implement it
                #os.system(("sudo ifconfig lo 127.0.0.{}  netmask 255.0.0.0").format(randint(0,50))) 
       
        #Save the content of the page 
        art=open("article"+str(i+1)+'.html',"wb")
        art.write(check.content)
        art.close()
    
    os.chdir("..")

 17%|█▋        | 3/18 [06:08<28:42, 114.83s/it]

Last well executed page is 366
Last well executed page is 366
Last well executed page is 366


 94%|█████████▍| 17/18 [25:00<01:28, 88.29s/it]


IndexError: list index out of range

## 1.3

In [65]:
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
def Parse2(tsv, soup):
    animeTitle,animeType,(animeNumEpisode),(releaseDate),(endDate),(animeNumMembers),(animeScore),(animeUsers),(animeRank),(animePopularity),animeDescription,(animeRelated),(animeCharacters),(animeVoices),(animeStaff)="","","","","","","","","","","","","","",""

    left_col=soup.find('div', {'style':"width: 225px"})
    l_div=left_col.find_all("div",{"class":"spaceit_pad"}) #we will iterate over this, more later
    
    start=""
    end=""
    
    #This is a for over the left column of the page, in which are listed most of the info we care about
    #going through the html like this speeds up the process of reading the single element
    for i in range(len(l_div)):
        
        if "Type" in l_div[i].getText():
            animeType = l_div[i].getText().replace("Type:","").replace("\n","").strip()
            #print(animeType)
            
        if "Episodes" in l_div[i].getText():
            animeNumEpisode = (l_div[i].getText().replace("Episodes:","").replace("\n","").strip())
            if animeNumEpisode=="Unknown":
                animeNumEpisode=""
            #print(animeNumEpisode)
        
        #this piece of code is probably not that readable but I had to adjust it in different times in order
        #to get as much info as possible since there were many different cases
        if "Aired" in l_div[i].getText():
            if len(l_div[i].getText().replace("Aired:","").replace("to","").strip().split())==6:
                start=" ".join(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split()[0:3])
                end=" ".join(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split()[3:6])
                start=datetime.strptime(start,"%b %d %Y")
                end=datetime.strptime(end,"%b %d %Y")
            if len(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split())==3 and ("?" not in l_div[i].getText().replace("Aired:","").replace("to","").strip().split()):
                if len(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split()[1])!=len(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split()[2]):
                    
                    start=" ".join(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split()[0:3])
                    start=datetime.strptime(start,"%b %d %Y")
                else:
                    start=""
            else:
                start=end=""
            
            releaseDate=start
            endDate=end
            #print(releaseDate)
            #print(endDate)
            
        #the i!=0 and "ookiku" exceptions are ad-hoc exceptions made for some animes
        #which contained info parsed in a different way from the rest of the group
        if ("Score" in l_div[i].getText()) and (i!=0) and ("Ookiku" not in l_div[i].getText()):
            if l_div[i].getText().replace("Score:","").replace(",","").split()[3]!="-":
                animeUsers = int(l_div[i].getText().replace("Score:","").replace(",","").split()[3])
                animeScore = float(l_div[i].getText().replace("Score:","").replace(",","").split()[0])
            #print(animeUsers)
            #print(animeScore)
        #if "Ranked" in l_div[i].getText():#19 >>>> conviene prenderlo da sopra il rank
            
        if "Popularity" in l_div[i].getText():  
            animePopularity = int(l_div[i].getText().replace("Popularity:","").strip().replace("#",""))
            #print(animePopularity)
    
        if "Members" in l_div[i].getText():
            animeNumMembers = int(l_div[i].getText().replace("Members:","").strip().replace(",",""))
            #print(animeNumMembers)
   
    
    description=soup.find("p",{"itemprop":"description"})
    animeDescription=description.getText().replace("\n","")
    #print(animeDescription)
    
    #THESE ARE THE SUGGESTED ONES
    #rel=[]
    #related=soup.find_all("li",{"class":"btn-anime","style":"width:90px"})
    #for i in range(len(related)):
    #    rel.append(related[i]["title"])
    #animeRelated = rel
    #print(animeRelated)
    
    rel=[]
    related=soup.find("table",{"class":"anime_detail_related_anime"})#.find_all("td",{"class":"borderClass"})
    if (related)!=None:
        #print(type(related),related)
        related=related.find_all("td",{"class":"borderClass"})
        for i in range(len(related)):
            if i%2!=0:
                rel.append(related[i].getText())
    animeRelated=rel
    
    
    chars=[]
    voices=[]
    staff=[]
    l=[]
    
    #As before, loop over the 2 tables in the html
    seed=soup.find_all("div",{"class":"detail-characters-list clearfix"})
    for i in range(len(seed)):
        if i==0:
            who=soup.find("div",{"class":"detail-characters-list clearfix"}).find_all("img")
            for i in range(len(who)):
                if i%2==0:
                    chars.append(",".join(who[i]["alt"].replace(" ","").split(",")))
                else:
                    voices.append(",".join(who[i]["alt"].replace(" ","").split(",")))
    
        if i==1:
            if (len(soup.find_all("div",{"class":"detail-characters-list clearfix"}))>1):
                for i in range(len(soup.find_all("div",{"class":"detail-characters-list clearfix"})[1].find_all("img"))):
                    staff.append(soup.find_all("div",{"class":"detail-characters-list clearfix"})[1].find_all("img")[i]["alt"])
                    staff.append((soup.find_all("div",{"class":"detail-characters-list clearfix"})[1].find_all("div", {"class":"spaceit_pad"}))[i].getText().strip())
                    
                for i in range(0,2*len(soup.find_all("div",{"class":"detail-characters-list clearfix"})[1].find_all("img")),2):
                    l.append(staff[i:i+2])
            
            
    animeCharacters = chars
    animeVoices = voices
    animeStaff = l   

    #print(animeStaff)
    
    
    animeTitle = soup.find("h1",{"class":"title-name h1_bold_none"}).getText()
    animeRank=soup.find("span",{"class":"numbers ranked"}).getText().replace("Ranked #","")
    #Here we exclude the missing information case
    if "N/A" not in animeRank:
        animeRank = int(animeRank)
    else:
        animeRank=""
    
    #write title and info in the TSV
    tsv.write("animeTitle"+"\t"+"animeType"+"\t"+"animeNumEpisode"+"\t"+"releaseDate"+"\t"+"endDate"+"\t"+"animeNumMembers"+"\t"+"animeScore"+"\t"+"animeUsers"+"\t"+"animeRank"+"\t"+"animePopularity"+"\t"+"animeDescription"+"\t"+"animeRelated"+"\t"+"animeCharacters"+"\t"+"animeVoices"+"\t"+"animeStaff"+"\n")
    tsv.write(animeTitle+"\t"+animeType+"\t"+str(animeNumEpisode)+"\t"+str(releaseDate)+"\t"+str(endDate)+"\t"+str(animeNumMembers)+"\t"+str(animeScore)+"\t"+str(animeUsers)+"\t"+str(animeRank)+"\t"+str(animePopularity)+"\t"+animeDescription+"\t"+str(animeRelated)+"\t"+str(animeCharacters)+"\t"+str(animeVoices)+"\t"+str(animeStaff))


In [70]:
#PARSING
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime

for page in tqdm(range(0,382)):
    
    start=50*page
    
    for i in range(start,start+50):
        #Open the articles of the page and save all the TSVs in a single directory
        os.chdir("Page"+str(page+1))
        art=open("article"+str(i+1)+'.html',"r")
        os.chdir("..")
        os.chdir("TSV")
        tsv=open("anime_"+str(i+1)+".tsv","w")
        
        soup=BeautifulSoup(art.read(), "html.parser") 
        
        #Just use the parsing function we developed earlier
        Parse2(tsv,soup)
        
        art.close()
        tsv.close()
        os.chdir("..")


100%|█████████▉| 381/382 [38:20<00:06,  6.04s/it] 


FileNotFoundError: [Errno 2] No such file or directory: 'article19084.html'

# Part 5

## 5.1

In [ ]:
#Pseudocode:
"""
A: array of length l of which we want to compute the maximum sum of non adjacent elements
def compute(A): 
    l=len(A)       
    gain=[]      declare 2 arrays: one to store the values of the gain at each iteration and the other 
    path=[]                        to use later to save only the elements of the gain we care about (the path chosen
                                   to get the sum)
    inc=exc=0    here we initialize the variables that will keep inside them the value of the 2 sums at each iteration:
                 one including and one excluding the single element 
                 
    for i in range(l):          then we just compute the value of the 2 sums at each iteration and save the gain 
        temp=inc                for each of them
        inc=max(inc,A[i]+exc)
        exc=temp            
        gain.append(inc-exc)
    
    for j in range(0,len(gain)-1,2):  here we compare the elements of the gain list and take only those of interest:
        if gain[j+1]>0:               if the gain is equal to 0 for that iteration the value is excluded "a priori"
            path.append(A[j+1])       then we impose the condition to only get alternate values for which the gain is
        else:                         greater than zero
            path.append(A[j])
    
    return sum(path),path
"""

## 5.2

In [676]:
#Implementation
#method developed starting from this: https://www.youtube.com/watch?v=UtGtF6nc35g
def compute(A):
    l=len(A)
    gain=[]
    path=[]
    inc=exc=0 #include, exclude --> compute value of sum including or excluding the i-th element

    for i in range(l):
        temp=inc
        inc=max(inc,A[i]+exc)
        exc=temp            
        gain.append(inc-exc)#save the gain

    
    for j in range(0,len(gain)-1,2):
        if gain[j+1]>0:
            path.append(A[j+1])
        else:
            path.append(A[j])
    
    return sum(path),path

In [677]:
A=[30, 40, 25, 50, 30, 20]
compute(A)

(110, [40, 50, 20])

Given an array A of length $n$ the complexity of the algorithm is $O(n)$ [excluding constants].\
It took me over 20 hours to come up with this solution (I had problems with the implementation), in which I tried different approaches to the problem:
1. I first tried looking at the list as a tree, but computing every possible case to then take the one with the highest output would lead me to a solution of order $O(n^2)$.
1. After this attempt I started searching on the internet and came across this video (https://www.youtube.com/watch?v=UtGtF6nc35g) in which the inclusion/exclusion method is explained. Making few changes to that algorithm I managed to get a result that is able to find the highest value solution along with the chosen elements of the starting array that seems to work in all the cases.